## Introdução: A Floresta e as Árvores


Imagine que nosso projeto é uma linha de montagem em uma fábrica. O cliente (usuário) chega com um pedido ("quero férias na praia"), e nosso trabalho é entregar um produto final (um roteiro completo). Cada chain é uma estação de trabalho especializada nessa linha de montagem.

Nosso fluxo é:

Input do Usuário: "Gosto de trekking em montanhas com clima ameno."

Estação 1 (chain_destino): Pega o interesse e decide o melhor destino.

Estação 2 (chain_restaurantes): Pega o destino e sugere restaurantes.

Estação 3 (chain_passeios): Pega o mesmo destino e sugere passeios.

Produto Final: Agrupa tudo e entrega ao usuário.

O orchestrador.py é o gerente da fábrica, garantindo que o produto passe de uma estação para a outra na ordem correta.

### LCEL - LangChain Expression Language

`prompt | modelo | parser = chain`

#### Chain Destino

In [10]:
# --- 1. Dependências e Configuração ---

from dotenv import find_dotenv, load_dotenv
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


In [2]:
# Carregar variáveis de ambiente (SUGESTÃO: Crie um arquivo .env na raiz do seu projeto)
load_dotenv(find_dotenv())

True

In [3]:
# --- 2. Definindo a Estrutura de Saída (O Molho) ---
# Usamos Pydantic para dizer à LLM EXATAMENTE como queremos a resposta.
# Isso garante um output consistente e fácil de usar no resto do programa.
class Destino(BaseModel):
    cidade: str = Field(description="Nome da cidade recomendada.")
    motivo: str = Field(description="Motivo da recomendação da cidade.")


# O Parser é quem "ensina" o modelo sobre a estrutura e depois "lê" a resposta do modelo.
parser = JsonOutputParser(pydantic_object=Destino)

In [4]:
# --- 3. Criando o Modelo (O Recheio) ---
# Aqui instanciamos o cérebro da operação.
# Melhor configuração do modelo
model = ChatOpenAI(
    model="gpt-4o-mini",  # Mais econômico para testes
    temperature=0.5,
    max_tokens=500,
    request_timeout=30
)

In [5]:
# --- 4. Criando o Template do Prompt (O Pão) ---
# Este é o nosso roteiro de instruções para o modelo.
# A variável {interesse} será preenchida pelo usuário.
# A variável {format_instructions} será preenchida pelo nosso Parser.
prompt = ChatPromptTemplate.from_template(
    """
    O usuário informou um interesse principal de atividade: "{interesse}".
    Sua tarefa é recomendar uma cidade no Brasil onde essa atividade
    seja muito popular e bem estruturada.

    Responda APENAS com o objeto JSON solicitado.

    {format_instructions}
    """,
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [6]:
# --- 5. Montando a Chain com LCEL (O Sanduíche!) ---
# A barra vertical "|" é o "e depois faça isso". É a essência do LCEL.
# 1. Pegue o input e coloque no prompt.
# 2. Envie o prompt preenchido para o modelo.
# 3. Pegue a resposta do modelo e passe para o parser.
chain_destino = prompt | model | parser

# --- 6. Executando a Chain ---
interesse_usuario = {"interesse": "trekking em chapadas"}
resultado = chain_destino.invoke(interesse_usuario)

print("--- Resultado da Chain Destino ---")
print(f"Tipo do resultado: {type(resultado)}")

--- Resultado da Chain Destino ---
Tipo do resultado: <class 'dict'>


In [7]:
import json

resultado_formatado = json.dumps(
    resultado,
    indent=4,
    ensure_ascii=False  # * Essencial para exibir caracteres como 'ç' e 'ã' corretamente
)
print(resultado_formatado)

{
    "cidade": "Chapada dos Guimarães",
    "motivo": "A cidade é famosa por suas trilhas deslumbrantes, belíssimas chapadas e uma infraestrutura bem desenvolvida para os amantes do trekking."
}


### Restaurante
___


In [8]:
class Restaurante(BaseModel):
    nome: str = Field(description="Nome do restaurante.")
    tipo: str = Field(description="Tipo de culinária do restaurante.")
    descricao: str = Field(description="Descrição do restaurante.")


class ListaRestaurantes(BaseModel):
    restaurantes: list[Restaurante]

In [11]:
parser = PydanticOutputParser(pydantic_object=ListaRestaurantes)

prompt = ChatPromptTemplate.from_template(
        """
    Você é um assistente especialista em gastronomia e trabalha para uma agência de viagens.

    Para a cidade {cidade}, sugira uma lista contendo:
    - 3 restaurantes de comida caseira de boa qualidade
    - 3 restaurantes mais sofisticados

    {format_instructions}
    """,
        partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [12]:
cidade = resultado["cidade"]
print(f"Cidade recomendada: {cidade}")

Cidade recomendada: Chapada dos Guimarães


In [13]:
chain = prompt | model | parser
resultado = chain.invoke({"cidade": cidade})
print(resultado)

restaurantes=[Restaurante(nome='Restaurante Bom Apetite', tipo='Comida Caseira', descricao='Um ambiente acolhedor que serve pratos caseiros feitos com ingredientes frescos da região, ideal para quem busca uma refeição saborosa e reconfortante.'), Restaurante(nome='Sabor do Campo', tipo='Comida Caseira', descricao='Restaurante familiar que oferece uma variedade de pratos típicos, preparados com carinho e receitas tradicionais, perfeito para almoços em família.'), Restaurante(nome='Casa da Comida', tipo='Comida Caseira', descricao='Um local charmoso que destaca a culinária regional, com pratos feitos em fogão a lenha e uma seleção de sobremesas caseiras irresistíveis.'), Restaurante(nome='Café do Cerrado', tipo='Gastronomia Sofisticada', descricao='Um restaurante elegante que combina sabores locais com técnicas contemporâneas, oferecendo uma experiência gastronômica única com um menu sazonal.'), Restaurante(nome='Ouro Verde', tipo='Gastronomia Sofisticada', descricao='Com uma vista deslu

In [14]:
print(type(resultado))

<class '__main__.ListaRestaurantes'>


In [15]:
# ... existing code ...
print("--- Lista de Restaurantes ---")
print(f"Cidade: {cidade}")
print()

# Restaurantes de comida caseira
print("🍽️  RESTAURANTES DE COMIDA CASEIRA:")
print("-" * 40)
for i, restaurante in enumerate(resultado.restaurantes[:3], 1):
    print(f"{i}. {restaurante.nome}")
    print(f"   Tipo: {restaurante.tipo}")
    print(f"   Descrição: {restaurante.descricao}")
    print()

# Restaurantes sofisticados
print("✨ RESTAURANTES SOFISTICADOS:")
print("-" * 40)
for i, restaurante in enumerate(resultado.restaurantes[3:], 1):
    print(f"{i}. {restaurante.nome}")
    print(f"   Tipo: {restaurante.tipo}")
    print(f"   Descrição: {restaurante.descricao}")
    print()

--- Lista de Restaurantes ---
Cidade: Chapada dos Guimarães

🍽️  RESTAURANTES DE COMIDA CASEIRA:
----------------------------------------
1. Restaurante Bom Apetite
   Tipo: Comida Caseira
   Descrição: Um ambiente acolhedor que serve pratos caseiros feitos com ingredientes frescos da região, ideal para quem busca uma refeição saborosa e reconfortante.

2. Sabor do Campo
   Tipo: Comida Caseira
   Descrição: Restaurante familiar que oferece uma variedade de pratos típicos, preparados com carinho e receitas tradicionais, perfeito para almoços em família.

3. Casa da Comida
   Tipo: Comida Caseira
   Descrição: Um local charmoso que destaca a culinária regional, com pratos feitos em fogão a lenha e uma seleção de sobremesas caseiras irresistíveis.

✨ RESTAURANTES SOFISTICADOS:
----------------------------------------
1. Café do Cerrado
   Tipo: Gastronomia Sofisticada
   Descrição: Um restaurante elegante que combina sabores locais com técnicas contemporâneas, oferecendo uma experiência 

### Passeios e ou Atividades Culturais
___

In [16]:
# * Estrutura de Dados
# * Chain 3 - Atrações
class Atracao(BaseModel):
    """Representa uma única atração turística.

    Attributes:
        nome (str): O nome da atração.
        descricao (str): Uma breve descrição da atração.

    """

    nome: str
    descricao: str


class ListaAtracoes(BaseModel):
    """Representa uma lista de atrações turísticas.

    Attributes:
        atracoes (list[Atracao]): Uma lista de objetos Atracao.

    """

    atracoes: list[Atracao]

In [17]:
parser = PydanticOutputParser(pydantic_object=ListaAtracoes)
prompt = ChatPromptTemplate.from_template(
        """
        Sugira 3 passeios culturais na cidade {cidade}.
        Para cada passeio, forneça:
        - nome: nome do passeio
        - descricao: descrição detalhada do passeio

        {format_instructions}
        """,
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
chain = prompt | model | parser
resultado = chain.invoke({"cidade": cidade})

In [18]:
print(type(resultado))

<class '__main__.ListaAtracoes'>


In [19]:
# ... existing code ...
print("=" * 60)
print(f"�� PASSEIOS CULTURAIS EM {cidade.upper()}")
print("=" * 60)

for i, atracao in enumerate(resultado.atracoes, 1):
    print(f"\n{i}. {atracao.nome}")
    print(f"   {atracao.descricao}")
    print("-" * 50)

�� PASSEIOS CULTURAIS EM CHAPADA DOS GUIMARÃES

1. Parque Nacional da Chapada dos Guimarães
   Um dos principais atrativos da região, o Parque Nacional da Chapada dos Guimarães é conhecido por suas impressionantes formações rochosas, cachoeiras e uma rica biodiversidade. Durante o passeio, os visitantes podem explorar trilhas que levam a mirantes com vistas deslumbrantes do pôr do sol, além de conhecer a famosa Cachoeira Véu de Noiva. O parque também é um ótimo lugar para observar aves e outros animais silvestres.
--------------------------------------------------

2. Cidade Histórica de Chapada dos Guimarães
   Este passeio oferece uma imersão na história e cultura local. Os visitantes podem caminhar pelas ruas de paralelepípedos da cidade, admirando a arquitetura colonial e os casarões históricos. O tour inclui visitas a igrejas, como a Igreja de Nossa Senhora do Rosário, e a Praça da Matriz, onde é possível conhecer mais sobre a história da fundação da cidade e sua importância na ép